<a href="https://colab.research.google.com/github/thofes/SJ_distance/blob/main/NN_Airtime_Landing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Connect GDrive
from google.colab import drive
drive.mount('/content.gdrive')

Mounted at /content.gdrive


In [ ]:
import json 
import numpy as np



with open('/content.gdrive/MyDrive/Manually_Labeled_01042022/dataset_distance.json', "r") as fp:
    data = json.load(fp)

X = np.array(data["keypoints"])
y = np.array(data["labels"])

print(X.shape)
print(y.shape)

(6701, 75)
(6701,)


In [ ]:
#Creating binary classification model 1575
!pip install -U tensorflow-addons

import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Activation, Dense, Conv1D, Dropout, MaxPooling1D, LSTM, Conv2D, Flatten,GlobalMaxPooling1D, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy, binary_crossentropy
from sklearn.utils import shuffle
import tensorflow_addons as tfa





def Ranger(sync_period=6,
           slow_step_size=0.5,
           learning_rate=0.001,
           beta_1=0.9,
           beta_2=0.999,
           epsilon=1e-7,
           weight_decay=0.,
           amsgrad=False,
           sma_threshold=5.0,
           total_steps=0,
           warmup_proportion=0.1,
           min_lr=0.,
           name="Ranger"):
    inner = tfa.optimizers.RectifiedAdam(learning_rate, beta_1, beta_2, epsilon, weight_decay, amsgrad, sma_threshold, total_steps, warmup_proportion, min_lr, name)
    optim = tfa.optimizers.Lookahead(inner, sync_period, slow_step_size, name)
    return optim



model = Sequential([
    
    Dense(units=750, input_shape=(75,1), activation='relu'), 
    Dense(units=1500, activation='relu'),                 
    Conv1D(filters=32, kernel_size=3, strides=3, padding='same',
           activation='relu'),
    Dropout(0.15),
    Conv1D(filters=64, kernel_size=3, strides=1, padding='same', 
           activation='relu'),
    Dropout(0.15),
    Flatten(),
    Dense(units=1, activation='sigmoid') 
])

save_callback = EarlyStopping(
    monitor="val_accuracy",
    min_delta=0,
    patience=50,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)
def scheduler(epoche, lr):
      if epoche < 2:
          return lr
      else:
          return lr*0.97
lr_scheduler = keras.callbacks.LearningRateScheduler(scheduler, verbose=1)
#model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=Ranger(), loss='binary_crossentropy', metrics=['accuracy'])


model.summary()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 75, 750)           1500      
                                                                 
 dense_3 (Dense)             (None, 75, 1500)          1126500   
                                                                 
 conv1d_2 (Conv1D)           (None, 25, 32)            144032    
                                                                 
 dropout_2 (Dropout)         (None, 25, 32)            0         
                                                                 
 conv1d_3 (Conv1D)           (None, 25, 64)            6208      
                                                                 
 dropout_3 (Dropout)         (None, 25, 64)            0         
                     

In [ ]:
history = model.fit(x=X, y=y, batch_size=40, validation_split = 0.1, epochs=1000, shuffle=True, verbose = 2, callbacks=[save_callback])

Epoch 1/1000
151/151 - 56s - loss: 0.6918 - accuracy: 0.5182 - val_loss: 0.6893 - val_accuracy: 0.5350 - 56s/epoch - 370ms/step
Epoch 2/1000
151/151 - 50s - loss: 0.6827 - accuracy: 0.5559 - val_loss: 0.6567 - val_accuracy: 0.6677 - 50s/epoch - 332ms/step
Epoch 3/1000
151/151 - 57s - loss: 0.6193 - accuracy: 0.6716 - val_loss: 0.5808 - val_accuracy: 0.7288 - 57s/epoch - 376ms/step
Epoch 4/1000
151/151 - 50s - loss: 0.5439 - accuracy: 0.7517 - val_loss: 0.5177 - val_accuracy: 0.7779 - 50s/epoch - 331ms/step
Epoch 5/1000
151/151 - 49s - loss: 0.4715 - accuracy: 0.7940 - val_loss: 0.4558 - val_accuracy: 0.8077 - 49s/epoch - 326ms/step
Epoch 6/1000
151/151 - 49s - loss: 0.4295 - accuracy: 0.8141 - val_loss: 0.4160 - val_accuracy: 0.8048 - 49s/epoch - 325ms/step
Epoch 7/1000
151/151 - 49s - loss: 0.3927 - accuracy: 0.8274 - val_loss: 0.3928 - val_accuracy: 0.8197 - 49s/epoch - 325ms/step
Epoch 8/1000
151/151 - 50s - loss: 0.3869 - accuracy: 0.8226 - val_loss: 0.3854 - val_accuracy: 0.8241 -

In [ ]:
import keras
from matplotlib import pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
model.save('/content.gdrive/MyDrive/NN_Model_20062022')

In [ ]:
#Load the model
#Creating binary classification model 1575
!pip install -U tensorflow-addons
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Activation, Dense, Conv1D, Dropout, MaxPooling1D, LSTM, Conv2D, Flatten,GlobalMaxPooling1D, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy, binary_crossentropy
from sklearn.utils import shuffle
import tensorflow_addons as tfa





def Ranger(sync_period=6,
           slow_step_size=0.5,
           learning_rate=0.001,
           beta_1=0.9,
           beta_2=0.999,
           epsilon=1e-7,
           weight_decay=0.,
           amsgrad=False,
           sma_threshold=5.0,
           total_steps=0,
           warmup_proportion=0.1,
           min_lr=0.,
           name="Ranger"):
    inner = tfa.optimizers.RectifiedAdam(learning_rate, beta_1, beta_2, epsilon, weight_decay, amsgrad, sma_threshold, total_steps, warmup_proportion, min_lr, name)
    optim = tfa.optimizers.Lookahead(inner, sync_period, slow_step_size, name)
    return optim


model = keras.models.load_model('/content.gdrive/MyDrive/Final/NN_Model_20062022')
model.summary()
